# 데이터 전처리

In [2]:
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.utils import shuffle
import numpy as np
import os
import shutil

## 데이터 읽기

In [2]:
data = pd.read_csv('listen_count.txt', sep=' ', header=None, dtype=str)
data.columns = ['uid', 'sid', 'cnt']
data['cnt'] = data['cnt'].astype(int)

In [3]:
try:
    os.mkdir('./parsed')
except FileExistsError:
    print("folder exist")
    shutil.rmtree('./parsed')
    os.mkdir('./parsed')

folder exist


## 유저 아이디 저장

유니크한 유저 아이디를 가져와서 저장합니다.

In [4]:
uids = sorted(data.uid.unique(), key=lambda x: int(x))

In [5]:
with open("./parsed/all_uids.txt", 'w') as f:
    print('\n'.join(uids), file=f)

In [6]:
n_user, n_song = len(data.uid.unique()), len(data.sid.unique())
data_len = len(data)

## 데이터셋 나누기

전체 데이터를 8:2 비율의 train/validation 데이터로 split합니다.

In [7]:
shuffled_data = shuffle(data)

In [8]:
ratio = 0.8

In [9]:
tr, val = shuffled_data[:int(data_len * ratio)], shuffled_data[int(data_len * ratio):]

In [10]:
tr_in_uids = set(tr.uid.unique())
tr_in_sids = set(tr.sid.unique())
val_filtered = val[val.uid.isin(tr_in_uids) & val.sid.isin(tr_in_sids)]

전처리 코드를 두번 작성하는 것을 막기 위해 저장되는 학습 파일은 실제 데이터([`listen_count.txt`](../listen_count.txt))와 같은 형태로 두는 것이 편리합니다.

In [11]:
tr.to_csv('./parsed/listen_count.txt', index=None, header=None, sep=' ')

validation 파일은 채점용 데이터([`jukebox.pdf`](../jukebox.pdf)에 기술되어 있는 형태)와 같은 형식으로 두는 것이 편리합니다.

In [12]:
save = val.groupby('uid').sid.apply(list).reset_index()
with open("./parsed/TEST_DATA.txt", 'w') as f:
    for uid, sids in zip(save.uid, save.sid):
        print("%s " % uid + ' '.join(sids), file=f)

데이터가 적절하게 잘 저장되어 있는지 한 번 더 확인합니다.

In [13]:
!echo "listen_count.txt"
!head ./parsed/listen_count.txt
!echo ""
!echo "TEST_DATA.txt"
!head ./parsed/TEST_DATA.txt
!echo ""

listen_count.txt
11377 7184 13
1852 206 3
19145 1692 3
15629 585 2
15821 5003 1
16033 2242 1
13306 2920 1
16734 2488 2
19990 23 5
9208 3913 1

TEST_DATA.txt
1 96 57 101 65 10 58 97 31 47 37 11 20 44 103 35 94 4 6 33 50 19
10 305 335 259 263 347 292 289 328 307 321 297 311 286 303 294 202 339 99 323 332 315 356 309 277
100 4547 4564 4475 4503 4502 420 4507 4501 1903 4576 4505 4504 4522 4533 4491 4570 4510 4556 4225 4521 4481 4542 4527 4506 4499 4520 4552 890 687 4523 4483 4544
1000 3595 3585 201
10000 1338 7336 5327 5256 1285 2821 4233 2517 545 2369 1172 4882 4299 427 1042 6979 1194 379 2953 433 137 5253 2494 2981 477
10001 4643
10002 200
10004 2966
10005 3723
10006 5826

